**Importing Libraries and Data**

In [ ]:
#Importing libraries
import pandas as pd
import statsmodels.api as sm
from sklearn.metrics import root_mean_squared_error
from statsmodels.tsa.stattools import acf
import matplotlib.pyplot as plt
import prophet
import numpy as np
import warnings
import gc
import os
import sys

#Ignoring warnings
warnings.filterwarnings("ignore")

#Reading in data and extracting unique account values
BoostedTransactions = pd.read_csv("Boosted Transaction Dataset.csv")
AccountIDs = BoostedTransactions["AccountId"].values
UniqueAccounts = set(AccountIDs)

#Converting transaction dates into months
BoostedTransactions["TransactionDate"] = pd.to_datetime(BoostedTransactions["TransactionDate"])
BoostedTransactions["TransactionDate"] = BoostedTransactions["TransactionDate"].dt.to_period("M").dt.to_timestamp()

#Grouping transactions by account and month and summating each grouping's net transaction amount
GroupedSumOfTransactionsByAccountAndMonth = BoostedTransactions.groupby(by=["AccountId", "TransactionDate"]).agg(["sum"])["Amount"]
print(GroupedSumOfTransactionsByAccountAndMonth)
GroupedSumOfTransactionsByAccountAndMonth = GroupedSumOfTransactionsByAccountAndMonth.reset_index()

                                                          sum
AccountId                            TransactionDate         
0003a5ae-0c77-4372-b44d-882ef9874a28 2019-05-01      -5541.31
                                     2019-06-01      -3601.46
                                     2019-07-01       3151.74
                                     2019-08-01      -4801.13
                                     2019-09-01       3890.05
...                                                       ...
fff7f00c-c869-4310-b705-4503538f5ecf 2020-03-01       2974.84
                                     2020-04-01        382.93
                                     2020-05-01        166.15
                                     2020-06-01       -550.89
                                     2020-07-01      -2721.18

[10995 rows x 1 columns]


**Key Prophet Loop**

In [ ]:
#Creating models and forecasting for all accounts
for i in UniqueAccounts:
    print(i)
    try:
        #Selecting only the selected account's time series and ensuring there are no gaps
        ForecastingTimeSeries = GroupedSumOfTransactionsByAccountAndMonth[GroupedSumOfTransactionsByAccountAndMonth["AccountId"] == i][["TransactionDate", "sum"]]

        ForecastingTimeSeries = ForecastingTimeSeries.set_index("TransactionDate")

        ForecastingTimeSeries = ForecastingTimeSeries.resample("MS").sum()

        ForecastingTimeSeries = ForecastingTimeSeries.reset_index()

        #Calculating the cutoff point between training and testing sets
        TrainEnd = int(0.7 * len(ForecastingTimeSeries))

        #Splitting the data into training and testing sets and adjusting column names in each
        TrainData = ForecastingTimeSeries[:TrainEnd]
        TestData = ForecastingTimeSeries[TrainEnd:]

        TrainData.columns = ["ds", "y"]
        TestData.columns = ["ds", "y"]

        #Declaring and training the model
        Model = prophet.Prophet()
        Model.fit(TrainData)

        #Performing a test forecast
        Forecast = Model.predict(TestData)

        #Calculating error rates and appending them into the errors CSV
        try:
          RMSE = root_mean_squared_error(Forecast["yhat"], TestData["y"])
          SI = abs(RMSE/abs(TestData["y"]).mean())
          AccountError = pd.DataFrame({"AccountID" : [i],
                                        "RMSE" : [RMSE],
                                        "SI" : [SI]})
          AccountError.to_csv("Prophet Errors (Month-by-Month).csv", mode='a', header=not os.path.exists("Prophet Errors (Month-by-Month).csv"), index=False)
        #Where a ValueError occurs, the loop is broken
        except ValueError:
          print("ValueError encountered")
          break

        #Forecasting the next 12 months
        Future = Model.make_future_dataframe(periods=12, freq="MS")
        FutureForecast = Model.predict(Future)

        FutureForecastTail = FutureForecast.tail(12)
        FutureForecastValues = FutureForecastTail["yhat"].values

        #Calculating highest amount, lowest amount and difference and appending them into the forecasts CSV file
        HighestAmount = FutureForecastValues.max()
        LowestAmount = FutureForecastValues.min()

        if np.isnan(HighestAmount) or np.isnan(LowestAmount):
          HighestAmount = 0
          LowestAmount = 0
        else:
          pass

        Difference = HighestAmount - LowestAmount

        ForecastData = pd.DataFrame({"AccountID" : [i],
                                     "Highest Amount" : [HighestAmount],
                                     "Lowest Amount" : [LowestAmount],
                                     "Difference" : [Difference]})

        ForecastData.to_csv("Prophet Forecasts (Month-by-Month).csv", mode='a', header=not os.path.exists("Prophet Forecasts (Month-by-Month).csv"), index=False)

        print("Processed account number",i)
    #Where an error occurs, the user is informed about it
    except Exception as e:
        print(f"Error encountered processing account number {i}.")
        ErrorType, ErrorObject, ErrorTraceback = sys.exc_info()

        ErrorFilename = os.path.split(
            ErrorTraceback.tb_frame.f_code.co_filename
        )[1]

        ErrorMessage = str(e)

        ErrorLineNumber = ErrorTraceback.tb_lineno

        print(f'Exception Type: {ErrorType}')

        print(f'Exception Filename: {ErrorFilename}')

        print(f'Exception Line Number: {ErrorLineNumber}')

        print(f'Exception Message: {ErrorMessage}')
        break
    #Deleting model data to conserve RAM
    finally:
        del Model, Forecast, TrainData, TestData, ForecastingTimeSeries
        gc.collect()


16460


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 7.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpg5jmmc3_/2brjbk4z.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpg5jmmc3_/y02rkgkp.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=86245', 'data', 'file=/tmp/tmpg5jmmc3_/2brjbk4z.json', 'init=/tmp/tmpg5jmmc3_/y02rkgkp.json', 'output', 'file=/tmp/tmpg5jmmc3_/prophet_modelkscybd05/prophet_model-20250807124253.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
12:42:53 - cmdstanpy - INFO - Chain [1] start processing
IN

Processed account number 16460
c4325b8f-693c-4605-b685-360e36c31697
Processed account number c4325b8f-693c-4605-b685-360e36c31697


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 7.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpg5jmmc3_/e2avmzzg.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpg5jmmc3_/u8d713q4.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=10952', 'data', 'file=/tmp/tmpg5jmmc3_/e2avmzzg.json', 'init=/tmp/tmpg5jmmc3_/u8d713q4.json', 'output', 'file=/tmp/tmpg5jmmc3_/prophet_model7gne9yhg/prophet_model-20250807124254.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
12:42:54 - cmdstanpy - INFO - Chain [1] start processing
IN

17f16911-4247-4728-a435-f4bf2f48e27b
Processed account number 17f16911-4247-4728-a435-f4bf2f48e27b


...

f083de1d-fd0a-4022-adda-0a5daa130b8f
Processed account number f083de1d-fd0a-4022-adda-0a5daa130b8f


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 3.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpg5jmmc3_/9s5_2jsk.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpg5jmmc3_/zbsz0qjs.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=5694', 'data', 'file=/tmp/tmpg5jmmc3_/9s5_2jsk.json', 'init=/tmp/tmpg5jmmc3_/zbsz0qjs.json', 'output', 'file=/tmp/tmpg5jmmc3_/prophet_modelnl6d3w3g/prophet_model-20250807124743.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
12:47:43 - cmdstanpy - INFO - Chain [1] start processing
INF

bb1082d7-b3ba-4f07-bc40-7e9936c75b1e
Processed account number bb1082d7-b3ba-4f07-bc40-7e9936c75b1e


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 7.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpg5jmmc3_/h4009nnr.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpg5jmmc3_/1ahbkkpo.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=16379', 'data', 'file=/tmp/tmpg5jmmc3_/h4009nnr.json', 'init=/tmp/tmpg5jmmc3_/1ahbkkpo.json', 'output', 'file=/tmp/tmpg5jmmc3_/prophet_model1gw6i_7e/prophet_model-20250807124743.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
12:47:43 - cmdstanpy - INFO - Chain [1] start processing
IN

5d466bfa-1922-473a-9f7c-1ddc671f41c0
Processed account number 5d466bfa-1922-473a-9f7c-1ddc671f41c0


**Calculating Error Statistics**

In [ ]:
#Calculating averaged error statistics
Errors = pd.read_csv("Prophet Errors (Month-by-Month).csv")
MeanRMSE = Errors["RMSE"].mean()
print("Mean RMSE:",MeanRMSE)
MedianRMSE = Errors["RMSE"].median()
print("Median RMSE:",MedianRMSE)
MeanSI = Errors["SI"].mean()
print("Mean SI:",MeanSI)
MedianSI = Errors["SI"].median()
print("Median SI:",MedianSI)

Mean RMSE: 25731.801586564492
Median RMSE: 4760.757842073419
Mean SI: 4.812886094180353
Median SI: 1.8204998091130915
